# 로지스틱 회귀분석 실습

python의 scikit-learn 패키지를 이용해 정규화 회귀분석을 직접 실행해보자.

### 데이터 불러오기 및 분할하기

로지스틱 회귀분석 실습을 위해 기존의 예측용 데이터셋이 아닌 분류용 데이터셋을 사용해 보자. 이번에 사용할 cancer dataset은 유방암 분류에 대한 데이터이다. 독립 변수는 30개이며, 종속 변수는 양성과 악성의 두 가지 범주를 가지고 있다. 자세한 사항은 [이곳](https://scikit-learn.org/stable/datasets/index.html#breast-cancer-wisconsin-diagnostic-dataset)에서 확인할 수 있다.

In [1]:
from sklearn import datasets
import numpy as np
data = datasets.load_breast_cancer()

x = data['data']
y = data['target']

print(x.shape, y.shape)

(569, 30) (569,)


In [2]:
from sklearn.model_selection import train_test_split

# 데이터 나누기 - 6:2:2 비율
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=1)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.25, random_state=1)

print(x_train.shape, x_val.shape, x_test.shape)

(341, 30) (114, 30) (114, 30)


### 모델 학습 및 평가하기

[LogisticRegression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html?highlight=logisticregression#sklearn.linear_model.LogisticRegression) 에 대한 자세한 설명은 링크를 참조하자. 설명을 읽어보면 선형 회귀분석과 마찬가지로 각종 regularization을 제공함을 알 수 있다.

In [3]:
from sklearn.linear_model import LogisticRegression
logistic = LogisticRegression()
logistic.fit(x_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

분류 문제의 경우 confusion matrix와 정확도를 이용해 성능을 확인해 볼 수 있다.
아래와 같이 training, validation, test의 성능을 구해 보자.

In [4]:
from sklearn.metrics import accuracy_score, confusion_matrix

pred_train = logistic.predict(x_train)
print(f'training accuracy: {accuracy_score(y_train, pred_train)}')
print(confusion_matrix(y_train, pred_train))

training accuracy: 0.9618768328445748
[[122   8]
 [  5 206]]


In [5]:
pred_val = logistic.predict(x_val)
print(f'validation accuracy: {accuracy_score(y_val, pred_val)}')
print(confusion_matrix(y_val, pred_val))

validation accuracy: 0.9649122807017544
[[37  3]
 [ 1 73]]


In [6]:
pred_test = logistic.predict(x_test)
print(f'test accuracy: {accuracy_score(y_test, pred_test)}')
print(confusion_matrix(y_test, pred_test))

test accuracy: 0.956140350877193
[[37  5]
 [ 0 72]]


앞서 등장한 경고 문구에 대해 알아보자. 

> ConvergenceWarning: lbfgs failed to converge (status=1): STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.


수업 시간에 배웠듯이, 로지스틱 회귀분석의 경우는 선형 회귀분석과 다르게 미분으로 간단히 최적의 매개변수를 구할 수 없다. 따라서 적절한 최적화 방법을 적용해야 하는데, 위 경고문은 이 최적화 과정이 계산 횟수의 제한으로 인해 완료되지 못하고 도중에 멈추었다는 뜻이다. 횟수의 제한을 풀어 최적화를 완료해 보자.




In [7]:
# 경고 문구 없이 로지스틱 회귀분석 학습 완료하기
logistic10000 = LogisticRegression(max_iter=10000)
logistic10000.fit(x_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=10000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [9]:
# 학습한 모델의 training, validation, test 정확도 알아보기
new_pred_train = logistic10000.predict(x_train)
print(f'accuracy: {accuracy_score(y_train, new_pred_train)}')
print(confusion_matrix(y_train, new_pred_train))

print('-----------------------------------')
new_pred_val = logistic10000.predict(x_val)
print(f'accuracy: {accuracy_score(y_val, new_pred_val)}')
print(confusion_matrix(y_val, new_pred_val))

print('-----------------------------------')
new_pred_test = logistic10000.predict(x_test)
print(f'accuracy: {accuracy_score(y_test, new_pred_test)}')
print(confusion_matrix(y_test, new_pred_test))

accuracy: 0.9648093841642229
[[123   7]
 [  5 206]]
-----------------------------------
accuracy: 0.9649122807017544
[[37  3]
 [ 1 73]]
-----------------------------------
accuracy: 0.9473684210526315
[[37  5]
 [ 1 71]]
